In [1]:
#Importing Libraries
import pandas as pd
import os
import tensorflow as tf
from keras import backend as K
from tensorflow.keras.preprocessing import image_dataset_from_directory
from keras.models import  Model
from keras.layers import Input, Add, Dense, Activation, ZeroPadding2D, BatchNormalization, Conv2D, GlobalAveragePooling2D,DepthwiseConv2D

In [3]:
#Extracting data
data_path = '../input/image-data'
work_path = '../work'

In [4]:
#Train and Test Data
train_path = os.path.join(work_path, 'train')
test_path = os.path.join(work_path, 'test1')

In [5]:
#Dataframe for images
train_df = pd.DataFrame({'image_name':os.listdir(train_path)})
train_df['label'] =train_df['image_name'].apply(lambda x: x.split('.')[0])

In [8]:
# Defining Variance Scaling Initializer
initializer = tf.keras.initializers.VarianceScaling(
scale=0.1, mode='fan_in', distribution='uniform')

In [9]:
#Defining Identity Block
def identity_block(X, f, filters, stage, block):
   
    conv_name_base = 'conv' + str(stage) + block + '_branch'
    bn_name_base = 'bn' + str(stage) + block + '_branch'
    F1,F3 = filters

    X_shortcut = X
   
    X = Conv2D(filters=F1, kernel_size=(1, 1), strides=(1, 1), padding='valid', name=conv_name_base + '2a', kernel_initializer=initializer)(X)
    X = BatchNormalization(axis=3, name=bn_name_base + '2a')(X)
    X = Activation('relu')(X)

    X = DepthwiseConv2D(kernel_size=(f, f), strides=(1, 1), padding='same', name=conv_name_base + '2b', kernel_initializer=initializer)(X)
    X = BatchNormalization(axis=3, name=bn_name_base + '2b')(X)
    X = Activation('relu')(X)

    X = Conv2D(filters=F3, kernel_size=(1, 1), strides=(1, 1), padding='valid', name=conv_name_base + '2c', kernel_initializer=initializer)(X)
    X = BatchNormalization(axis=3, name=bn_name_base + '2c')(X)

    X = Add()([X, X_shortcut])# SKIP Connection
    X = Activation('relu')(X)

    return X

In [10]:
#Defining Main Block
def main_block(X, f, filters, stage, block):
   
    conv_name_base = 'conv' + str(stage) + block + '_branch'
    bn_name_base = 'bn' + str(stage) + block + '_branch'
    F1, F3 = filters
   
    X = Conv2D(filters=F1, kernel_size=(1, 1), strides=(1, 1), padding='valid', name=conv_name_base + '2a', kernel_initializer=initializer)(X)
    X = BatchNormalization(axis=3, name=bn_name_base + '2a')(X)
    X = Activation('relu')(X)

    X = ZeroPadding2D((3, 3))(X)
    X = DepthwiseConv2D(kernel_size=(f, f), strides=(1, 1), padding='same', name=conv_name_base + '2b', kernel_initializer=initializer)(X)
    X = BatchNormalization(axis=3, name=bn_name_base + '2b')(X)
    X = Activation('relu')(X)

    X = Conv2D(filters=F3, kernel_size=(1, 1), strides=(1, 1), padding='valid', name=conv_name_base + '2c', kernel_initializer=initializer)(X)
    X = BatchNormalization(axis=3, name=bn_name_base + '2c')(X)

    return X

In [11]:
# Defining the Google Teachable Machine that makes use of identity and main block
def GoogleTeachableMachine(input_shape=(224, 224, 3)):

    X_input = Input(input_shape)

    X = ZeroPadding2D((3, 3))(X_input)

    X = Conv2D(16, (3, 3), strides=(2, 2), name='conv1', kernel_initializer=initializer)(X)
    X = BatchNormalization(axis=3, name='bn_conv1')(X)
    X = Activation('relu')(X)
    X = DepthwiseConv2D(16, (3, 3), name='conv2', kernel_initializer=initializer)(X)
    X = BatchNormalization(axis=3, name='bn_conv2')(X)
    X = Activation('relu')(X)
    X = Conv2D(16, (3, 3), strides=(2, 2), name='conv3', kernel_initializer=initializer)(X)
    X = BatchNormalization(axis=3, name='bn_conv3')(X)
    
    X = main_block(X, 3, [48,8], stage=2, block='b')
    X = identity_block(X, 3, [48,8], stage=2, block='c')
    
    X = main_block(X, 3, [48,16], stage=3, block='b')
    X = identity_block(X, 3, [96,16], stage=3, block='c')
    
    X = identity_block(X, 3, [96,16], stage=4, block='b')
    
    X = main_block(X, 3, [96,24], stage=5, block='b')
    X = identity_block(X, 3, [144,24], stage=5, block='c')
    
    X = identity_block(X, 3, [144,24], stage=6, block='b')
    
    X = identity_block(X, 3, [144,24], stage=7, block='b')
    
    X = main_block(X, 3, [144,32], stage=8, block='b')
    X = identity_block(X, 3, [192,32], stage=8, block='c')
    
    X = identity_block(X, 3, [192,32], stage=9, block='b')
    
    X = main_block(X, 3, [192,56], stage=10, block='b')
    X = identity_block(X, 3, [336,56], stage=10, block='c')
    
    X = identity_block(X, 3, [336,56], stage=11, block='b')
    
    X = main_block(X, 3, [336,112], stage=12, block='b')
    
    X = Conv2D(1280, (3, 3), strides=(2, 2), name='conv13', kernel_initializer=initializer)(X)
    X = BatchNormalization(axis=3, name='bn_conv13')(X)
    X = Activation('relu')(X)

    X = GlobalAveragePooling2D()(X)
    
    model = Model(inputs=X_input, outputs=X, name='ResNet50')

    return model

In [12]:
#Defining the main model
base_model = GoogleTeachableMachine(input_shape=(224, 224, 3))

2022-04-04 03:13:09.466276: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:937] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2022-04-04 03:13:09.583370: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:937] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2022-04-04 03:13:09.584537: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:937] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2022-04-04 03:13:09.586668: I tensorflow/core/platform/cpu_feature_guard.cc:142] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags

In [13]:
#Adding the Dense and Output layers
headModel = base_model.output
headModel=Dense(100, activation='relu', name='dense1',kernel_initializer=initializer)(headModel)
headModel=Dense(1,activation='softmax', name='dense2',kernel_initializer=initializer)(headModel)

In [14]:
model = Model(inputs=base_model.input, outputs=headModel)

In [15]:
model.summary()

Model: "model"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            [(None, 224, 224, 3) 0                                            
__________________________________________________________________________________________________
zero_padding2d (ZeroPadding2D)  (None, 230, 230, 3)  0           input_1[0][0]                    
__________________________________________________________________________________________________
conv1 (Conv2D)                  (None, 114, 114, 16) 448         zero_padding2d[0][0]             
__________________________________________________________________________________________________
bn_conv1 (BatchNormalization)   (None, 114, 114, 16) 64          conv1[0][0]                      
______________________________________________________________________________________________

In [16]:
# Displaying all the layers in the model
for layer in model.layers:
    print(layer, layer.trainable)

<keras.engine.input_layer.InputLayer object at 0x7f0cd1712e90> True
<keras.layers.convolutional.ZeroPadding2D object at 0x7f0cd173bc50> True
<keras.layers.convolutional.Conv2D object at 0x7f0cd030c390> True
<keras.layers.normalization.batch_normalization.BatchNormalization object at 0x7f0cd0c66610> True
<keras.layers.core.Activation object at 0x7f0cd0cf45d0> True
<keras.layers.convolutional.DepthwiseConv2D object at 0x7f0cd0c66950> True
<keras.layers.normalization.batch_normalization.BatchNormalization object at 0x7f0cd0cfc990> True
<keras.layers.core.Activation object at 0x7f0cd0e6ad90> True
<keras.layers.convolutional.Conv2D object at 0x7f0cd1bdc4d0> True
<keras.layers.normalization.batch_normalization.BatchNormalization object at 0x7f0cd0e80810> True
<keras.layers.convolutional.Conv2D object at 0x7f0cd0e76f90> True
<keras.layers.normalization.batch_normalization.BatchNormalization object at 0x7f0cd0e0c4d0> True
<keras.layers.core.Activation object at 0x7f0cd0e19190> True
<keras.laye

In [17]:
epochs=3

In [18]:
image_size = (224, 224)
batch_size = 32
rand_seed = 42
val_split = 0.2

In [ ]:
#class_names=['cat', 'dog'] -- Can be diiferent depending on given dataset
class_names=['cat','dog']

In [19]:
# Training dataset
train_ds = image_dataset_from_directory(
    directory=train_path, 
    class_names=class_names,
    batch_size=batch_size,
    image_size=image_size,
    seed=rand_seed, 
    validation_split=val_split,
    subset='training'
)

Found 25000 files belonging to 2 classes.
Using 20000 files for training.


In [20]:
# Compiling the model
model.compile(
    optimizer='adam',
    loss='binary_crossentropy',
    metrics=['accuracy']
)

In [21]:
# Validation dataset
val_ds = image_dataset_from_directory(
    directory=train_path,
    class_names=class_names,
    batch_size=batch_size,
    image_size=image_size,
    seed=rand_seed,
    validation_split=val_split,
    subset='validation'
)

Found 25000 files belonging to 2 classes.
Using 5000 files for validation.


In [1]:
# Fitting the model
hist=model.fit(train_ds,epochs=epochs,validation_data=val_ds)

Epoch 1/3
2022-04-04 03:13:25.113187: I tensorflow/compiler/mlir/mlir_graph_optimization_pass.cc:185] None of the MLIR Optimization Passes are enabled (registered 2)
2022-04-04 03:13:27.593325: I tensorflow/stream_executor/cuda/cuda_dnn.cc:369] Loaded cuDNN version 8005

625/625 [==============================] - 195s 287ms/step - loss: 0.5503 - accuracy: 0.5981 - val_loss: 0.9054 - val_accuracy: 0.6076
Epoch 2/3
625/625 [==============================] - 177s 283ms/step - loss: 0.4533 - accuracy: 0.7981 - val_loss: 0.6145 - val_accuracy: 0.8045
Epoch 3/3
625/625 [==============================] - 179s 285ms/step - loss: 0.2321 - accuracy: 0.9281 - val_loss: 0.4242 - val_accuracy: 0.9314
